<a href="https://colab.research.google.com/github/hvmarco/PV-feasibility/blob/main/project_marcohernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PV feasibility analysis
This is the project for the course <i>Python Foundations for Spatial Analysis (2024)</i> taught by Ujaval Gandhi and: Vigna Purohi at Spatial Thoughts.

First we need to install some additional libraries and import them.

In [1]:
#install and import necessary packages
%%capture
# use %%capture to prevent printing the output of the cell
!pip install numpy-financial
!pip install pvlib[optional]

import numpy as np
import numpy_financial as npf #to use financial functions in numpy
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import pvlib    #toolbox for simulating the performance of photovoltaic energy systems https://pvlib-python.readthedocs.io/en/stable/index.html#


Since we will be using some files with the data, we clone the repository in the colab environment to import them

In [7]:
%%capture
#clone the repository to include the files needed
!git clone https://github.com/hvmarco/PV-feasibility.git

In [6]:
from pvlib import pvsystem
#import the components databases
moddb = pvsystem.retrieve_sam(path = 'PV-feasibility/CEC Modules.csv')
invdb = pvsystem.retrieve_sam(path = 'PV-feasibility/CEC Inverters.csv')

#select the module and inverter we will be using
pv_module = moddb['Trina_Solar_TSM_670DEG21C_20']
inverter = invdb['Sungrow_Power_Supply_Co___Ltd___SG250HX_US__800V_']


/usr/local/lib/python3.10/dist-packages/pvlib/pvsystem.py:2085: UserWarning: Original names contain 29 duplicate(s).
  warnings.warn('Original names contain %d duplicate(s).' % n_duplicates)
/usr/local/lib/python3.10/dist-packages/pvlib/pvsystem.py:2089: UserWarning: Normalized names contain 29 duplicate(s).
  warnings.warn(


# New Section

In [8]:
pv_module

Manufacturer    Trina Solar
Technology        Mono-c-Si
Bifacial                  1
STC                 671.055
PTC                   632.8
A_c                    3.08
Length                  NaN
Width                   NaN
N_s                      66
I_sc_ref              18.55
V_oc_ref               46.3
I_mp_ref              17.43
V_mp_ref               38.5
alpha_sc           0.006307
beta_oc           -0.106953
T_NOCT                 44.6
a_ref               1.60651
I_L_ref             18.5766
I_o_ref                 0.0
R_s                0.155978
R_sh_ref             108.82
Adjust              6.72645
gamma_r              -0.303
BIPV                      N
Version          2023.10.31
Date             11/16/2022
Name: Trina_Solar_TSM_670DEG21C_20, dtype: object